In [3]:
# pip install google-generativeai

  Obtaining dependency information for google-generativeai from https://files.pythonhosted.org/packages/3b/2c/a82d94e8829d64e1dae2e28cfe8da6010888a8e2dcb942901d4197f73546/google_generativeai-0.5.2-py3-none-any.whl.metadata
  Obtaining dependency information for google-ai-generativelanguage==0.6.2 from https://files.pythonhosted.org/packages/ef/a3/06fa48fcadf2c1f3b32f2e4950333a3bfa16a1a22d9183f64cdd0d359773/google_ai_generativelanguage-0.6.2-py3-none-any.whl.metadata
  Obtaining dependency information for google-api-core from https://files.pythonhosted.org/packages/86/75/59a3ad90d9b4ff5b3e0537611dbe885aeb96124521c9d35aa079f1e0f2c9/google_api_core-2.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for google-api-python-client from https://files.pythonhosted.org/packages/de/27/60ad979de67b96ac4bea93c86d4251f7d438754919f6222c43f18c45486c/google_api_python_client-2.126.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for proto-plus<2.0.0dev,>=1.22.3 from h

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 1.6 MB/s eta 0:00:0000:010:010m
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.62.0
    Uninstalling grpcio-1.62.0:
      Successfully uninstalled grpcio-1.62.0
Note: you may need to restart the kernel to use updated packages.


In [64]:
from datetime import date
import re
import json

today = date.today()

# dd/mm/YY
td = today.strftime("%d/%m/%Y")

In [31]:
# user_input = "Person A: Hey, are you free to chat about the project timeline for a bit?\nPerson B: Sure, I have some time now. What's up?\nPerson A: I was thinking we should solidify some deadlines for the next few phases. Do you have your calendar handy?\nPerson B: Yeah, just a sec... Okay, I'm looking at it now.\nPerson A: Great. So, for the research phase, do you think we can wrap that up by the end of next week?\nPerson B: Hmm, that might be a little tight. I have a couple of other deadlines that week. Could we push it to the following Monday?\nPerson A: That works for me. And then for the design phase, are we still aiming for a two-week turnaround?\nPerson B: Ideally, yes. But let's see how the research phase goes and adjust if needed. We might need to add a buffer day or two.\nPerson A: Sounds good. We can have a check-in next Friday to see where we're at and finalize the design phase deadline then.\nPerson B: Perfect. Should I put a placeholder on the calendar for that check-in?\nPerson A: Yes, please do! And let's add a tentative deadline for the design phase as well, just so we have a target to aim for.\nPerson B: Done and done. Anything else we need to discuss for now?\nPerson A: I think that covers it for now. Thanks for taking the time to chat!\nPerson B: No problem, talk to you soon!"

progress = {'task': [],
           'category': [],
           'progress':[]}

In [233]:
"""
At the command line, only need to run once to install the package via pip:

$ pip install google-generativeai
"""
def parse_string(string_to_parse):
    # Find the start and end index of the JSON part
    start_index = string_to_parse.find('{')
    end_index = string_to_parse.rfind('}') + 1

    # Extract the JSON part from the string
    json_string = string_to_parse[start_index:end_index]

    # Remove leading and trailing whitespace and newlines
    json_string = json_string.strip()

    # Remove the initial 'json' label and surrounding newlines
    json_string = json_string.replace('```json', '').replace('```', '').strip()

    # Split the string by commas to get key-value pairs
    pairs = json_string.split(',')
    
    # Initialize an empty dictionary to store key-value pairs
    parsed_dict = {}

    # Iterate over key-value pairs and split them by colons to extract keys and values
    for pair in pairs:
        key, value = pair.split(':')
        # Remove surrounding whitespace and quotes from the key and value
        key = key.strip().strip('"')
        value = value.strip().strip('"')
        # Add the key-value pair to the dictionary
        parsed_dict[key] = value
    return parsed_dict

import google.generativeai as genai

genai.configure(api_key="")

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 100,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]
#     try:
#         system_instruction1 = "Given the current task: {}. As a personal assistant, your task is to inquire about the progress on the current tasks and to gather information about any new tasks along with their deadlines. It's important to approach this with empathy and encouragement, especially if the user seems tired.".format(str(h2))
#     except:
system_instruction1 = "As a personal assistant, your task is to inquire about the progress on the current tasks and to gather information about any new tasks along with their deadlines. It's important to approach this with empathy and encouragement, especially if the user seems tired."
system_instruction2 = "Given that today is {}. Extract the due date of every task mentioned, and output in the json format.".format(td)

model1 = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction1,
                              safety_settings=safety_settings)

model2 = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction2,
                              safety_settings=safety_settings)

convo1 = model1.start_chat(history=[])
convo2 = model2.start_chat(history=[])

def gemini(input_text):
    

    
    #conversation
    response1 = convo1.send_message(input_text)
    print(response1.text)
    
    response2 = model2.generate_content(input_text)
#     progress.update(parse_string(response2.text))
    
#     return progress
    print(response2.text)

# print(convo.last.text)

In [234]:
progress = gemini('Hi')

Hi! How are you doing today?  Just checking in to see how things are going with your current tasks and if there's anything new on your plate.  No worries if you're feeling a bit overwhelmed, we can figure things out together.  😊 

```json
{
  "due_dates": []
}
``` 



In [235]:
progress = gemini('I have assignment that dues in two days, and another exam on the same day')

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).